# 🧑‍🍳 A kitchen that responds to your “I’m hungry” is more than feasible

# 🔥 Get a kernel ready

This is second nature to you now.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from IPython.display import display, Markdown

kernel = sk.Kernel()

useAzureOpenAI = False

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenaicompletion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel.add_text_embedding_generation_service("azureopenaiembedding", AzureTextEmbedding("text-embedding-ada-002", endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("openaicompletion", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    kernel.add_text_embedding_generation_service("openaiembedding", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

## 🦄 Find the right function for me, and then use it

We're going to be doing two things:

1. Let the `ActionPlanner` know what skills are available to it.
2. Give it a goal formed as an `ask`.
3. Have it find the most similar function that can get the job done.
4. Make it so.

In [ ]:
from semantic_kernel.planning import ActionPlanner

planner = ActionPlanner(kernel)

from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

ask = "What is the sum of 110 and 990?"

print(f"🧲 Finding the most similar function available to get that done...")
plan = await planner.create_plan_async(goal=ask)
print(f"🧲 The best single function to use is `{plan._skill_name}.{plan._function.name}`")

result = await plan.invoke_async()
display(Markdown(f"## ✨ For the ask '{ask}'"))
print(f"✨ The generated result is: {result}")

Note that the `ActionPlanner` isn't much of a planner — it's more of an automatic function selector. If you want to imagine a plan as a sequence of steps, and at each step using a different automatically selected function, let's keep going.

## 🦄 Generate a multi-step plan and execute it

Let's show you the `SequentialPlanner` which generates a plan more like you expect. As of August 2023 our most advanced C#-based `StepwisePlanner` is not available in our Python package, but it soon will be. How's it different? Well, it's a lot more sophisticated — and more importantly, it is more clever at giving you what you have `ask`-ed for :+).

You can trace the steps of the `SequentialPlanner` by setting `trace_resultsp` to `True`.

In [ ]:
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.core_skills.text_skill import TextSkill

plugins_directory = "./plugins-sk"
writer_skill = kernel.import_semantic_skill_from_directory(plugins_directory, "LiterateFriend")

# create an instance of sequential planner.
planner = SequentialPlanner(kernel)

# the ask for which the sequential planner is going to find a relevant function.
ask = """
Tomorrow is Valentine's day. I need to come up with a poem. Translate the poem to French."""

# ask the sequential planner to identify a suitable function from the list of functions available.
plan = await planner.create_plan_async(goal=ask)

# ask the sequential planner to execute the identified function.
result = await plan.invoke_async()

for index, step in enumerate(plan._steps):
    print(f"✅ Step {index+1} used function `{step._function.name}`")

trace_resultp = False

if trace_resultp:
    print("Longform trace:\n")
    for index, step in enumerate(plan._steps):
        print("Step:", index)
        print("Description:",step.description)
        print("Function:", step.skill_name + "." + step._function.name)
        print("Input vars:", step._parameters._variables)
        print("Output vars:", step._outputs)
        if len(step._outputs) > 0:
            print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

display(Markdown(f"## ✨ Generated result from the ask: {ask}\n\n---\n" + str(result)))

# 🔖 There are a variety of limitations to using the planner in August of 2023 in terms of number of tokens required and model preference that we can expect to slowly vanish over time. For simple tasks, this Planner-based approach is unusually powerful. It takes full advantage of both COMPLETION and SIMILARITY in a truly magical way.

![](assets/twodimensions.png)